In [1]:
import pandas as pd
books = pd.read_csv('books_with_categories.csv')

In [10]:
from transformers import  pipeline
classifier = pipeline('text-classification', model = 'j-hartmann/emotion-english-distilroberta-base', top_k =None)
classifier('I fear this')

Device set to use cpu


[[{'label': 'fear', 'score': 0.9889454245567322},
  {'label': 'neutral', 'score': 0.0036433173809200525},
  {'label': 'sadness', 'score': 0.002623705193400383},
  {'label': 'surprise', 'score': 0.0015594592550769448},
  {'label': 'anger', 'score': 0.0014993511140346527},
  {'label': 'joy', 'score': 0.0009505611378699541},
  {'label': 'disgust', 'score': 0.0007782159373164177}]]

In [4]:
books['description'][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [12]:
classifier(books['description'][0])

[[{'label': 'fear', 'score': 0.6548399925231934},
  {'label': 'neutral', 'score': 0.1698525995016098},
  {'label': 'sadness', 'score': 0.11640939861536026},
  {'label': 'surprise', 'score': 0.02070068009197712},
  {'label': 'disgust', 'score': 0.019100721925497055},
  {'label': 'joy', 'score': 0.015161462128162384},
  {'label': 'anger', 'score': 0.003935154061764479}]]

In [11]:
classifier(books['description'][0].split('.'))

[[{'label': 'surprise', 'score': 0.729602038860321},
  {'label': 'neutral', 'score': 0.14038607478141785},
  {'label': 'fear', 'score': 0.06816229224205017},
  {'label': 'joy', 'score': 0.04794258251786232},
  {'label': 'anger', 'score': 0.009156374260783195},
  {'label': 'disgust', 'score': 0.002628477755934},
  {'label': 'sadness', 'score': 0.0021221644710749388}],
 [{'label': 'neutral', 'score': 0.44937077164649963},
  {'label': 'disgust', 'score': 0.27359139919281006},
  {'label': 'joy', 'score': 0.10908306390047073},
  {'label': 'sadness', 'score': 0.09362738579511642},
  {'label': 'anger', 'score': 0.040478240698575974},
  {'label': 'surprise', 'score': 0.02697017788887024},
  {'label': 'fear', 'score': 0.0068790484219789505}],
 [{'label': 'neutral', 'score': 0.6462157964706421},
  {'label': 'sadness', 'score': 0.24273352324962616},
  {'label': 'disgust', 'score': 0.04342266544699669},
  {'label': 'surprise', 'score': 0.028300534933805466},
  {'label': 'joy', 'score': 0.014211485

In [25]:
import numpy as np
emotion_labels = ['anger', 'disgust','fear','joy','sadness','surprise','neutral']
isbn = []
emotion_scores = { label:[] for label in emotion_labels }

def calculate_max_emotion_score(predictions):
    per_emotion_scores = { label:[] for label in emotion_labels }
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x:x['label'])
        for index,label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]['score'])
    return {label: np.max(scores) for label,scores in per_emotion_scores.items()}

In [26]:
from tqdm import tqdm
for i in tqdm(range(len(books))):
    isbn.append(books['isbn13'][i])
    sentences = books['description'][i].split('.')
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_score(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|████████████████████████████████████████████████████████████████████████████| 5197/5197 [7:52:54<00:00,  5.46s/it]


In [27]:
emotion_df = pd.DataFrame(emotion_scores)
emotion_df['isbn13']=isbn

In [28]:
emotion_df

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273591,0.928168,0.932798,0.646216,0.967158,0.729602,9780002005883
1,0.612619,0.348285,0.942528,0.704422,0.887940,0.111690,0.252546,9780002261982
2,0.064134,0.104007,0.972321,0.767238,0.549477,0.111690,0.078765,9780006178736
3,0.351484,0.150722,0.360706,0.251881,0.732685,0.111690,0.078765,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884390,0.475880,0.078765,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.148208,0.030643,0.919165,0.255172,0.853721,0.980877,0.030656,9788172235222
5193,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765,9788173031014
5194,0.009997,0.009929,0.339218,0.947779,0.375754,0.066685,0.057625,9788179921623
5195,0.064134,0.104007,0.459268,0.759456,0.951104,0.368111,0.078765,9788185300535


In [29]:
books = pd.merge(books, emotion_df, on='isbn13')

In [30]:
books

,Unnamed: 0,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,...,title and subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273591,0.928168,0.932798,0.646216,0.967158,0.729602
1,1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,NaN,0.612619,0.348285,0.942528,0.704422,0.887940,0.111690,0.252546
2,3,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767238,0.549477,0.111690,0.078765
3,4,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,NaN,0.351484,0.150722,0.360706,0.251881,0.732685,0.111690,0.078765
4,5,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",NaN,0.081412,0.184495,0.095043,0.040564,0.884390,0.475880,0.078765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,6802,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,NaN,0.148208,0.030643,0.919165,0.255172,0.853721,0.980877,0.030656
5193,6803,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,...,Journey to the East,9788173031014 This book tells the tale of a ma...,NaN,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765
5194,6804,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,NaN,0.009997,0.009929,0.339218,0.947779,0.375754,0.066685,0.057625
5195,6805,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.064134,0.104007,0.459268,0.759456,0.951104,0.368111,0.078765


In [31]:
books.to_csv('books_with_emotion.csv', index=False)